In [5]:
# make sure it is done before starting to train
!cd .. && python setup.py install && pip install pytorch_lightning==1.5

running install
running bdist_egg
running egg_info
writing ncps.egg-info\PKG-INFO
writing dependency_links to ncps.egg-info\dependency_links.txt
writing requirements to ncps.egg-info\requires.txt
writing top-level names to ncps.egg-info\top_level.txt
reading manifest file 'ncps.egg-info\SOURCES.txt'
adding license file 'LICENSE'
writing manifest file 'ncps.egg-info\SOURCES.txt'
installing library code to build\bdist.win-amd64\egg
running install_lib
running build_py
copying ncps\__init__.py -> build\lib\ncps
copying ncps\datasets\icra2020_lidar_collision_avoidance.py -> build\lib\ncps\datasets
copying ncps\datasets\utils.py -> build\lib\ncps\datasets
copying ncps\datasets\__init__.py -> build\lib\ncps\datasets
copying ncps\keras\cfc.py -> build\lib\ncps\keras
copying ncps\keras\cfc_cell.py -> build\lib\ncps\keras
copying ncps\keras\ltc.py -> build\lib\ncps\keras
copying ncps\keras\ltc_cell.py -> build\lib\ncps\keras
copying ncps\keras\mm_rnn.py -> build\lib\ncps\keras
copying ncps\kera

C:\Users\VID\anaconda3\envs\lnn_slaf2\lib\site-packages\setuptools\_distutils\cmd.py:66: SetuptoolsDeprecationWarning: setup.py install is deprecated.
!!

        ********************************************************************************
        Please avoid running ``setup.py`` directly.
        Instead, use pypa/build, pypa/installer or other
        standards-based tools.

        See https://blog.ganssle.io/articles/2021/10/setup-py-deprecated.html for details.
        ********************************************************************************

!!
  self.initialize_options()
C:\Users\VID\anaconda3\envs\lnn_slaf2\lib\site-packages\setuptools\_distutils\cmd.py:66: EasyInstallDeprecationWarning: easy_install command is deprecated.
!!

        ********************************************************************************
        Please avoid running ``setup.py`` and ``easy_install``.
        Instead, use pypa/build, pypa/installer or other
        standards-based tools.

 

In [5]:
from run import main

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
main()

Epoch 0:  44%|████▍     | 69/157 [01:35<02:02,  1.39s/it, loss=50, v_num=8, train_loss=0.281]

C:\Users\VID\Desktop\GitHub\ncps-slaf\slaf-project\run.py:25: DtypeWarning: Columns (15,16,17,18,19) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(path)


GPU available: True, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
C:\Users\VID\anaconda3\envs\lnn_slaf2\lib\site-packages\pytorch_lightning\trainer\trainer.py:1566: UserWarning: GPU available but not used. Set the gpus flag in your trainer `Trainer(gpus=1)` or script `--gpus=1`.
  rank_zero_warn(
C:\Users\VID\anaconda3\envs\lnn_slaf2\lib\site-packages\pytorch_lightning\trainer\configuration_validator.py:118: UserWarning: You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.
  rank_zero_warn("You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.")

  | Name  | Type | Params
-------------------------------
0 | model | LTC  | 2.1 K 
-------------------------------
1.7 K     Trainable params
416       Non-trainable params
2.1 K     Total params
0.009     Total estimated model params size (MB)
C:\Users\VID\anaconda3\envs\lnn_slaf2\lib\site-packages\pytorch_lightning\callbacks\model_checkpoint.p

Epoch 1:   0%|          | 0/7 [00:00<?, ?it/s, loss=2.87e+04, v_num=10, train_loss=0.533]        

In [15]:
x_features = x_features.view(1, -1, 12).to("cuda")

NameError: name 'x_features' is not defined

In [16]:
with torch.no_grad():
    prediction = ltc_model(x_features)[0].cpu().numpy()

NameError: name 'torch' is not defined

In [5]:
import torch
torch.cuda.empty_cache()

In [18]:
import pytorch_lightning as pl
from src.model import SequenceLearner
from ncps.torch import LTC

In [19]:
ltc_model.rnn_cell._params

NameError: name 'ltc_model' is not defined

In [20]:
model = SequenceLearner.load_from_checkpoint("log/default/version_5/epoch=0-step=311-v14.ckpt")
model.model.to("cuda")
model.model.eval()

FileNotFoundError: [Errno 2] No such file or directory: 'C:/Users/VID/Desktop/GitHub/ncps-slaf/slaf-project/slaf-project/checkpoints/epoch=0-step=311-v14.ckpt'

In [33]:
x_features

tensor([[[0.2900, 0.2262, 0.2055,  ..., 1.0000, 0.0000, 0.0000],
         [0.2594, 0.1932, 0.1697,  ..., 0.0000, 0.0000, 0.0000],
         [0.2398, 0.1805, 0.1433,  ..., 0.0000, 0.0000, 0.0000],
         ...,
         [0.5665, 0.3527, 0.4628,  ..., 0.0000, 0.0000, 0.0000],
         [0.5246, 0.3314, 0.4441,  ..., 0.0000, 0.0000, 0.0000],
         [0.4662, 0.3093, 0.3949,  ..., 0.0000, 0.0000, 0.0000]]],
       device='cuda:0')

In [34]:
# predict with the model
y_hat = model.model(x_features)

In [35]:
y_hat

(tensor([[[0.2006],
          [0.3179],
          [0.3485],
          [0.3368],
          [0.3201],
          [0.3162],
          [0.3363],
          [0.3957],
          [0.4668],
          [0.5157],
          [0.5344],
          [0.5338],
          [0.5278],
          [0.5155],
          [0.4922],
          [0.4679],
          [0.4582],
          [0.4750],
          [0.5078],
          [0.5271],
          [0.5344],
          [0.5348],
          [0.5264],
          [0.5041],
          [0.4795],
          [0.4660],
          [0.4463],
          [0.4388],
          [0.4573],
          [0.5038],
          [0.5604],
          [0.6024],
          [0.6313],
          [0.6532],
          [0.6687],
          [0.6774],
          [0.6802],
          [0.6794],
          [0.6774],
          [0.6756],
          [0.6761],
          [0.6817],
          [0.6905],
          [0.6969],
          [0.6987],
          [0.6963],
          [0.6893],
          [0.6766],
          [0.6595],
          [0.6444],
